In [15]:
from pyspark import SparkConf, SparkContext
from pyspark.sql import SQLContext

In [16]:
# from pyspark import SparkContext, SparkConf
conf=SparkConf().setMaster("local").setAppName("Most Popular Movies")
sc=SparkContext(conf=conf)

In [17]:
# Creating movie_id and movie_name dictionary using pandas

import pandas as pd
df = pd.read_table("/mnt/home/edureka_294428/u.item", sep="|", encoding='ISO-8859-1', header=None)
df = df[[0, 1]]
movies_dict = dict(zip(df[0], df[1]))

# Creating a broadcast variable of dictionary
movies_lookup = sc.broadcast(movies_dict)

In [19]:
# Creating movie data rdd
movie_rdd=sc.textFile("hdfs://nameservice1/user/edureka_294428/u.data")

print(movie_rdd.take(2))

# Creating Key-Value paired rdd
movies_kv_rdd=movie_rdd.map(lambda x:(int(x.split()[1]),1))

[u'196\t242\t3\t881250949', u'186\t302\t3\t891717742']


In [20]:
# Putting reduce operation on Key-value pairs
movies_count=movies_kv_rdd.reduceByKey(lambda x, y: x + y)
movies_count.take(2)

[(1, 452), (2, 131)]

In [21]:
# Sorting the movie by keys
sorted_pairs = movies_count.sortBy(lambda x: x[1], ascending=False)
print(sorted_pairs.take(5))

[(50, 583), (258, 509), (100, 508), (181, 507), (294, 485)]


In [22]:
# Above doesn't make much of a sense unless we replace movie ids by movie names
# Here comes the use of Broadcast variable we created earlier

sorted_names=sorted_pairs.map(lambda x:(movies_lookup.value[x[0]],x[1]))
sorted_names.take(3)

[(u'Star Wars (1977)', 583), (u'Contact (1997)', 509), (u'Fargo (1996)', 508)]

In [23]:
#Top 10 Most Popular Movies
sorted_names.take(10)

[(u'Star Wars (1977)', 583),
 (u'Contact (1997)', 509),
 (u'Fargo (1996)', 508),
 (u'Return of the Jedi (1983)', 507),
 (u'Liar Liar (1997)', 485),
 (u'English Patient, The (1996)', 481),
 (u'Scream (1996)', 478),
 (u'Toy Story (1995)', 452),
 (u'Air Force One (1997)', 431),
 (u'Independence Day (ID4) (1996)', 429)]

In [14]:
sc.stop()